In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import os
import pickle
from collections import Counter, defaultdict
from itertools import combinations

import numpy as np
import scipy as sp
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import linalg
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

import numpy as np
from sklearn.manifold import TSNE
import glob

from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
df= pd.read_csv("/data/shruti/conspirecovery/reddit/initial/network/adjacency_matrix_full.csv", header=0)
df.head()

,Unnamed: 0,n1,n2,weight
0,0,WeAreTheMusicMakers,spotify,151
1,1,StarWars,crafts,47
2,2,milwaukee,politics,166
3,3,AutoNewspaper,NDTVauto,120278
4,4,AllThingsCrypto,CryptoCurrency,12283


In [3]:
subreddit_embeddings = pd.read_csv("/data/shruti/conspirecovery/reddit/initial/network/subreddit_embeddings.csv", 
                                   header=0)

In [29]:
subreddit_embeddings.sample(5)

,Unnamed: 0,subreddit,0,1,2,3,4,5,6,7,...,190,191,192,193,194,195,196,197,198,199
8488,8488,ThoughtfulLibertarian,-0.004974,-0.039726,-0.069413,-0.004075,-0.046677,-0.008120,-0.270648,0.058881,...,-0.098277,-0.052073,0.074216,-0.104735,-0.216387,-0.016425,0.014455,-0.042698,-0.380754,0.084575
483,483,FUI,-0.077603,-0.108817,-0.030679,-0.023587,-0.082516,-0.017375,-0.010298,0.051383,...,-0.064489,0.010908,-0.037561,0.097757,-0.005361,0.003621,-0.029947,0.039166,-0.009536,0.056442
7803,7803,ineedmoresleep,-0.079711,0.097255,-0.024423,-0.022075,-0.045685,-0.059573,-0.014690,-0.011875,...,0.043736,-0.060929,0.096707,0.011644,-0.262788,-0.055270,-0.019865,-0.067165,-0.495822,0.131088
1773,1773,GearsOfWar,0.188288,-0.039489,-0.122030,0.125572,-0.098298,-0.123557,-0.051776,0.023100,...,0.001257,0.405895,-0.314048,0.254220,-0.202375,-0.476919,0.103324,0.129840,0.101623,0.492943
5865,5865,wrestling,-0.145298,0.124521,-0.145719,-0.010300,0.094027,-0.108915,0.017067,-0.086742,...,-0.091454,0.214832,-0.139456,0.137051,-0.089028,0.244955,-0.184709,-0.302290,0.086499,0.417634


In [8]:
collist = list(subreddit_embeddings)[2:]

In [10]:
sublist = subreddit_embeddings['subreddit'].tolist()
svdMat = np.array(subreddit_embeddings[collist].values)

In [32]:
subb = 'climateskeptics'
climate_similarity = pd.DataFrame(columns=['subreddit', 'cos_dist'])
climate_index = sublist.index(subb)
climate_vector = svdMat[climate_index, :].reshape(-1, 1)
for s in sublist:
    s_index = sublist.index(s)
    s_vector = svdMat[s_index, :].reshape(-1, 1)
    coss = cosine(climate_vector, s_vector)
    climate_similarity = climate_similarity.append({'subreddit':s, 'cos_dist':coss}, ignore_index=True)
    
climate_similarity.sort_values(by=['cos_dist']).head(50)

,subreddit,cos_dist
5914,climateskeptics,0.000000
5317,GlobalWarming,0.222368
6484,climatechange,0.251667
8005,GlobalClimateChange,0.294969
5461,climate_science,0.297624
488,LockCarbon,0.328524
5302,ClimateHeadlineFraud,0.338298
294,StopFossilFuels,0.346954
911,climate,0.352044
5195,climatepolicy,0.359794
